# Setup

In [1]:
import subprocess
import tempfile
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
import random

### SET CWD TO REPO ROOT
os.chdir(Path.cwd().parent)
print("CWD now:", Path.cwd())

from rtl.py_lmul import lmul
from rtl.lmul_tester import BatchLMULTester

CWD now: /workspaces/LMUL-Hardware-Acceleration
✓ Batch tester ready


# Utilities

In [2]:
def float_to_bf16(f: float) -> int:
    """Convert float32 to bfloat16 (16-bit)"""
    if np.isnan(f):
        return 0x7FC0
    if np.isinf(f):
        return 0xFF80 if f < 0 else 0x7F80
    f = np.clip(f, -3.4e38, 3.4e38)
    f32_bits = struct.unpack('>I', struct.pack('>f', np.float32(f)))[0]
    bf16_bits = (f32_bits >> 16) & 0xFFFF
    return bf16_bits

def bf16_to_float(bf16: int) -> float:
    """Convert bfloat16 to float32"""
    f32_bits = (bf16 & 0xFFFF) << 16
    return struct.unpack('>f', struct.pack('>I', f32_bits))[0]

print("✓ BF16 utilities ready")



✓ BF16 utilities ready


# Tests

In [8]:
N = 10000
test_data = []
for _ in range(N):
    a_float = np.random.uniform(-10000, 10000)
    b_float = np.random.uniform(-10, 10)
    a_bf16 = float_to_bf16(a_float)
    b_bf16 = float_to_bf16(b_float)
    test_data.append((a_bf16, b_bf16, a_float, b_float))

In [9]:
lmul_results = []
pylmul_results = []
py_results = []

tester = BatchLMULTester()
test_pairs = [(a_bf16, b_bf16) for a_bf16, b_bf16, _, _ in test_data]
lmul_res, _ = tester.test_batch(test_pairs, verbose=False)
lmul_results = [bf16_to_float(res) for res in lmul_res]

for a_bf16, b_bf16, a_float, b_float in test_data:
    pylmul_res = bf16_to_float(lmul(a_bf16, b_bf16))
    py_res = a_float * b_float
    pylmul_results.append(pylmul_res)
    py_results.append(py_res)

lmul_results = np.array(lmul_results).astype(np.float32)
pylmul_results = np.array(pylmul_results).astype(np.float32)
py_results = np.array(py_results).astype(np.float32)

In [14]:
def test_results(lmul_results, pylmul_results, py_results, test_data):
    """Measure the accuracy of lmul_results against pylmul_results and py_results."""
    value_range = np.max(test_data) - np.min(test_data)
    
    if value_range == 0:
        print("Range of values is zero; cannot compute percentage differences.")
        return
    
    avg_diff_pylmul = np.mean(np.abs(lmul_results - pylmul_results))
    avg_diff_py = np.mean(np.abs(lmul_results - py_results))
    
    percentage_diff_pylmul = (avg_diff_pylmul / value_range) * 100
    percentage_diff_py = (avg_diff_py / value_range) * 100
    
    print(f"Average difference between lmul_results and pylmul_results: {avg_diff_pylmul:.6f} "
          f"({percentage_diff_pylmul:.2f}% of range)")
    print(f"Average difference between lmul_results and py_results: {avg_diff_py:.6f} "
          f"({percentage_diff_py:.2f}% of range)")

test_results(lmul_results, pylmul_results, py_results, test_data)

Average difference between lmul_results and pylmul_results: 0.000000 (0.00% of range)
Average difference between lmul_results and py_results: 913.799377 (1.51% of range)
